In [ ]:
import logging

from IPython.core.display import display, HTML

from scripts.main import *
from scripts.retrieve_data import *
from scripts.extract_data import *

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

### Load the data

In [ ]:
config = load_config()
config['main']['start_date'] = '2019-01-07'
config['main']['end_date'] = '2019-04-26'
    
# create the path where the data for the current config would be stored
day_period_str = '{}_{}'.format(config['main']['start_date'], config['main']['end_date']).replace('-', '')
studies_save_path = 'data/studies/studies_{}.pkl'.format(day_period_str)

df = pd.read_pickle(studies_save_path)

### Clean the data a bit and keep only some part of the data

In [ ]:
df['Machine'] = df['Machine Group'].str.replace('NoCT', '')
df = df[df['Machine'] != 'mixed cases']
logging.info('Machines: {}'.format(', '.join(set(df['Machine']))))
df

In [ ]:
df